In [1]:
import os 
import pandas as pd
import re

In [2]:
def numerical_sort(filename):
    return [int(x) if x.isdigit() else x for x in re.split(r'(\d+)', filename)]

def getFileList(folder):
    file_list = [folder+'/'+i for i in os.listdir(folder)]
    file_list = sorted(file_list, key=numerical_sort)

    return file_list

In [3]:
def results(file_list):
    results_df = pd.DataFrame()
    for file in file_list:
        df = pd.read_csv(file)
        df = df.iloc[:, 1:]
        results_df = pd.concat([results_df, df])

    return results_df.sort_values(by='test_r2', ascending=False)

In [28]:
t3_pandas = getFileList('../pandas/Results_nthresh_3/')
t3_pandas_df = results(t3_pandas)
t3_pandas_df.sort_values(by=['numDataPoints','test_r2'], ascending=[False, False])

,ICD,train_r2,test_r2,rmse,numDataPoints
0,I6359,0.836895,0.757321,0.136177,76015
0,Z3801,0.721827,0.447302,0.191407,70445
0,J189,0.739250,0.564649,0.244716,56823
0,I214,0.651465,0.415438,0.228688,44343
0,J9600,0.878951,0.706829,0.159035,43480
1,N179,0.679992,0.382933,0.225465,43185
3,N390,0.682863,0.436578,0.228710,41986
1,O80,0.757403,0.541435,0.210622,36843
0,I4891,0.668409,0.448522,0.226991,30702
1,Z3800,0.794827,0.374077,0.184225,30244


In [27]:
t3_dense = getFileList('../pandas/Results_nthresh_3_polars_10000/')
t3_dense_df = results(t3_dense)
t3_dense_df.sort_values(by=['numDataPoints','test_r2'], ascending=[False, False])

,ICD,train_r2,test_r2,rmse,numDataPoints
2,Z3800,0.609151,0.349670,0.163482,87787
1,A419,0.817408,0.637543,0.212193,66982
0,N179,0.651271,0.398312,0.217658,51602
1,I2510,0.821758,0.685638,0.220995,45743
1,O80,0.706916,0.533060,0.206021,44662
0,O700,0.650249,0.431166,0.192847,42850
0,O701,0.629916,0.389423,0.189457,42833
0,O3421,0.543097,0.258042,0.183710,42303
1,I5021,0.847237,0.636990,0.252858,30385
0,I5031,0.831802,0.621393,0.260167,30088


In [19]:
t3_dense_df[(t3_dense_df['numDataPoints'] > 10000) & (t3_dense_df['test_r2'] > 0.65)].sort_values(by=['numDataPoints','test_r2'],ascending=[False,False])

,ICD,train_r2,test_r2,rmse,numDataPoints
1,I6359,0.829970,0.734719,0.142705,76015
2,J9600,0.867169,0.683030,0.164396,43480
2,I5023,0.925535,0.701115,0.160310,25995
3,I2609,0.955789,0.775159,0.113045,19893
1,I120,0.972863,0.819341,0.113076,19571
2,I5033,0.952846,0.707591,0.151422,19228
3,J9620,0.969057,0.812585,0.124266,17354
2,M179,0.979074,0.809314,0.130626,14478
1,I10,0.991940,0.924063,0.069936,13632
2,R001,0.987341,0.790339,0.111663,12188


In [15]:
t3_dense_df.sort_values(by=['numDataPoints','test_r2'],ascending=[False,False]).head(20)

,ICD,train_r2,test_r2,rmse,numDataPoints
1,I6359,0.829970,0.734719,0.142705,76015
0,Z3801,0.716187,0.424072,0.193486,70445
0,J189,0.788243,0.571939,0.239422,56823
0,I214,0.645151,0.406624,0.234941,44343
2,J9600,0.867169,0.683030,0.164396,43480
4,N179,0.631579,0.369633,0.227456,43185
6,N390,0.725610,0.398141,0.231358,41986
2,O80,0.780372,0.520245,0.217057,36843
0,I4891,0.682917,0.407541,0.235097,30702
3,Z3800,0.741891,0.351621,0.188528,30244
